In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
from collections import defaultdict
import math

In [2]:
# format the data to be fed into the network
combine_cleaned = pd.read_csv('combine_cleaned.csv')
final_continuous = pd.read_csv('finalData.csv')
final_discrete = pd.read_csv('discreteData.csv')


# Choose dataset to use
data = final_discrete


In [3]:
# need to figure out how to add 'school', 'conference' back in. 
featuresStandard = ['heightinchestotal', 'weight', 'fortyyd', 'vertical', 'bench', 'twentyss', 'threecone', 'broad', 
            'games', 'rushingAtt', 'rushingYds', 'rushingAvg', 'rushingTD', 'passCmp',
            'passPct', 'passYds', 'passTD', 'passInt', 'passRate', 'recYds', 'recAtt', 'recTD', 'recAvg', 'soloTackles', 
            'tackleAssists', 'totalTackles', 'sacks', 'ints', 'intTDs', 'passDef', 'fumbles', 'fumblesForced']
featuresDiscrete = ['confACC','confPac-12','confUnknown','confSEC','confBig 12','confBig Ten','confAmerican',
            'confBig East','confPac-10','confMAC','confSun Belt','confMWC','confWAC','confCUSA','confInd','confSouthern','confMVC','confPac-8',
            'confBig West','confSWC','confSouthland','confBig 8','confSWAC','heightinchestotalNone','heightinchestotalQ1','heightinchestotalQ2',
            'heightinchestotalQ3','heightinchestotalQ4','weightNone','weightQ1','weightQ2','weightQ3','weightQ4','fortyydNone','fortyydQ1','fortyydQ2',
            'fortyydQ3','fortyydQ4','verticalNone','verticalQ1','verticalQ2','verticalQ3','verticalQ4','benchNone','benchQ1','benchQ2','benchQ3',
            'benchQ4','twentyssNone','twentyssQ1','twentyssQ2','twentyssQ3','twentyssQ4','threeconeNone','threeconeQ1','threeconeQ2','threeconeQ3',
            'threeconeQ4','broadNone','broadQ1','broadQ2','broadQ3','broadQ4','gamesNone','gamesQ1','gamesQ2','gamesQ3','gamesQ4','rushingAttNone',
            'rushingAttQ1','rushingAttQ2','rushingAttQ3','rushingAttQ4','rushingYdsNone','rushingYdsQ1','rushingYdsQ2','rushingYdsQ3','rushingYdsQ4',
            'rushingAvgNone','rushingAvgQ1','rushingAvgQ2','rushingAvgQ3','rushingAvgQ4','rushingTDNone','rushingTDQ1','rushingTDQ2','rushingTDQ3','rushingTDQ4'
            ,'passCmpNone','passCmpQ1','passCmpQ2','passCmpQ3','passCmpQ4','passPctNone','passPctQ1','passPctQ2','passPctQ3','passPctQ4','passYdsNone','passYdsQ1',
            'passYdsQ2','passYdsQ3','passYdsQ4','passTDNone','passTDQ1','passTDQ2','passTDQ3','passTDQ4','passIntNone','passIntQ1','passIntQ2','passIntQ3','passIntQ4','passRateNone',
            'passRateQ1','passRateQ2','passRateQ3','passRateQ4','recYdsNone','recYdsQ1','recYdsQ2','recYdsQ3','recYdsQ4','recAttNone','recAttQ1','recAttQ2','recAttQ3',
            'recAttQ4','recTDNone','recTDQ1','recTDQ2','recTDQ3','recTDQ4','recAvgNone','recAvgQ1','recAvgQ2','recAvgQ3','recAvgQ4','soloTacklesNone','soloTacklesQ1',
            'soloTacklesQ2','soloTacklesQ3','soloTacklesQ4','tackleAssistsNone','tackleAssistsQ1','tackleAssistsQ2','tackleAssistsQ3','tackleAssistsQ4','totalTacklesNone',
            'totalTacklesQ1','totalTacklesQ2','totalTacklesQ3','totalTacklesQ4','sacksNone','sacksQ1','sacksQ2','sacksQ3','sacksQ4','intsNone','intsQ1','intsQ2','intsQ3',
            'intsQ4','intTDsNone','intTDsQ1','intTDsQ2','intTDsQ3','intTDsQ4','passDefNone','passDefQ1','passDefQ2','passDefQ3','passDefQ4','fumblesNone','fumblesQ1',
            'fumblesQ2','fumblesQ3','fumblesQ4','fumblesForcedNone','fumblesForcedQ1','fumblesForcedQ2','fumblesForcedQ3','fumblesForcedQ4']


In [4]:
MAX_YEAR = 2015 
MIN_YEAR = 1999
TEST_YEAR = 2015
features = featuresDiscrete
shouldNormalize = True if features == featuresStandard else False 

In [5]:
# get player values/draft positions based on 'nfl_draft' data
def get_player_draft_info():
    valuesMap = {}
    pickMap = {}
    MAX_VALUE = 256 # number of draft picks + 1 (7 rounds * 32 picks + 32 compensatory + 1)
    for index, row in data.iterrows():
        primaryKey = (row['name'], row['year'])
        valuesMap[primaryKey] = (MAX_VALUE - row['pick'])
        pickMap[primaryKey] = row['pick']
    return valuesMap, pickMap

In [6]:
# Creates list of positions to partition by
positionsSet = set()
for position in data['pos']:
    positionsSet.add(position)
positions = list(positionsSet)

In [7]:
draftValues, draftPositions = get_player_draft_info()

In [24]:
# replace here with NEW dataset 
combineFeatures = data[['name', 'pos', 'year'] + features]

,name,pos,year,confACC,confPac-12,confUnknown,confSEC,confBig 12,confBig Ten,confAmerican,...,fumblesNone,fumblesQ1,fumblesQ2,fumblesQ3,fumblesQ4,fumblesForcedNone,fumblesForcedQ1,fumblesForcedQ2,fumblesForcedQ3,fumblesForcedQ4
0,JameisWinston,QB,2015,1,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
1,MarcusMariota,QB,2015,0,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
2,DanteFowler,LB,2015,0,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
3,AmariCooper,WR,2015,0,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
4,BrandonScherff,T,2015,0,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
5,LeonardWilliams,DE,2015,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
6,KevinWhite,WR,2015,0,0,0,0,1,0,0,...,1,0,0,0,0,1,0,0,0,0
7,VicBeasley,LB,2015,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
8,EreckFlowers,T,2015,0,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
9,ToddGurley,RB,2015,0,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0


In [9]:
def get_position_maps():
    positionMaps = {}
    for position in positions:
        positionMaps[position] = combineFeatures[combineFeatures['pos'] == position]
    return positionMaps

In [10]:
positionMaps = get_position_maps()

In [11]:
# build X matrix and Y values 
def build_data_arrays():
    X = defaultdict(list)
    Y = defaultdict(list)
    xTrain = defaultdict(list)
    yTrain = defaultdict(list)
    xTest = defaultdict(list)
    yTest = defaultdict(list)
    names = defaultdict(list)
    for position in positionMaps:
        for index, row in positionMaps[position].iterrows(): 
            X[position].append(row[3:])                  
            Y[position].append(draftValues[(row['name'], row['year'])])
            if row['year'] == TEST_YEAR:
                xTest[position].append(row[3:]) 
                yTest[position].append(draftValues[(row['name'], row['year'])])
                names[position].append(row['name'])
            if not row['year'] == TEST_YEAR:
                xTrain[position].append(row[3:])                  
                yTrain[position].append(draftValues[(row['name'], row['year'])])
    return X, Y, xTrain, yTrain, xTest, yTest, names

In [12]:
X, Y, xTrain, yTrain, xTest, yTest, names = build_data_arrays()

In [13]:
seed = 7
np.random.seed(seed)
predictor = linear_model.LinearRegression(fit_intercept=True, normalize=shouldNormalize, copy_X=True, n_jobs=1)

In [14]:
numFolds = 10
kFold = KFold(n_splits=numFolds, random_state=seed)
def eval_data():
    errorMap = {}
    for p in positions:
        print p, len(X[p])
        if len(X[p]) > numFolds:
            results = cross_val_score(predictor, np.array(X[p]), np.array(Y[p]), cv=kFold, scoring='mean_squared_error')
            errorMap[p] = (results.mean(), results.std())
            print 'Results[{}]: mean={} (std={}) Mean Squared Error'.format(p, results.mean(), results.std())
        else:
            print 'Position: {} has too little data ({} row(s))'.format(p, len(X[p]))
    return errorMap

In [15]:
# errors = eval_data()

In [16]:
# errors

In [17]:
# Map players to their true and predicted draft positions 
def get_relative_error(results, test_year):
    trueDraftMap = {}      # true pick number
    predictedDraftMap = {} # relative pick based on regression
    i = 0
    for index, row in results.iterrows():
        trueDraftMap[row['name']] = draftPositions[(row['name'], test_year)] #need primary key to get draft positions 
        predictedDraftMap[row['name']] = i
        i += 1
    
    # Compute absolute_error based on relative draft positions
    i = 0
    absolute_error = 0
    errors = {}
    for key, value in sorted(trueDraftMap.iteritems(), key=lambda (k,v): (v,k)):  
        errors[key] = math.fabs(i - predictedDraftMap[key]) 
        absolute_error += errors[key]
        i += 1
        
    #Add the errors to the results Dataframe (for visualization)
    results['error'] = 0.0
    for index, row in results.iterrows():
        results.at[index, 'error'] = errors[row['name']]

    #average the error by the total number of players
    return absolute_error / float(len(results))

In [18]:
scores = {}
relativeError = {} 
coefficients = {}
output = {}
for p in positions:
    if len(xTrain[p]) > 1 and len(xTest[p]) > 1:
        predictor.fit(np.array(xTrain[p]), np.array(yTrain[p]))
        coefficients[p] = pd.DataFrame(zip(features, predictor.coef_), columns = ['feature', 'coefficient']).sort_values(by=['coefficient'], ascending=False)
        prediction = predictor.predict(np.array(xTest[p])) 
        output[p] = pd.DataFrame(zip(names[p], prediction), columns = ['name', 'value']).sort_values(by=['value'], ascending=False)
        # scores[p] = predictor.score(np.array(xTest[p]), np.array(yTest[p]))
        scores[p] = (mean_squared_error(np.array(yTest[p]), np.array(prediction)),
                     r2_score(np.array(yTest[p]), np.array(prediction)))
        relativeError[p] = get_relative_error(output[p], TEST_YEAR)
    else:
        print 'Not enough data for position: {}'.format(p)

Not enough data for position: LS
Not enough data for position: K
Not enough data for position: P


In [19]:
scores

{'C': (2691.7142857142858, 0.45413083245730934),
 'DB': (4614.782608695652, 0.20154952808666826),
 'DE': (3624.1904761904761, 0.26922690037730468),
 'DT': (1.7842250739244422e+28, -3.5121798026796715e+24),
 'FB': (1.3836275691933287e+26, -6.4409069003209854e+22),
 'G': (3547.25, 0.067501532855538837),
 'LB': (5964.8194444444443, -0.15017541113635668),
 'QB': (11175.542635236468, -0.74955938173443259),
 'RB': (8.5741077491733894e+28, -1.9733540783784878e+25),
 'T': (300357.09999999998, -37.518135220016902),
 'TE': (7.2727548797018044e+30, -1.7148082870736767e+27),
 'WR': (1.3944259854796755e+28, -2.4177455911165319e+24)}

In [20]:
output

{'C':             name  value  error
 1      AliMarpet  196.0    1.0
 2   HronissGrasu  169.0    1.0
 4      MaxGarcia  169.0    2.0
 0  CameronErving  155.0    3.0
 5     AndyGallik  120.0    1.0
 3      ShaqMason   65.0    2.0
 6   AustinReiter   65.0    0.0, 'DB':                 name  value  error
 3         ByronJones  208.0    3.0
 21          JoshShaw  192.0   20.0
 37      QuandreDiggs  188.0   35.0
 29       DamianSwann  180.0   26.0
 1       KevinJohnson  172.0    3.0
 33     CharlesGaines  168.0   28.0
 9        RonaldDarby  168.0    3.0
 4   DamariousRandall  168.0    3.0
 0         TraeWaynes  164.0    8.0
 8           EricRowe  164.0    1.0
 14        PJWilliams  160.0    4.0
 2       MarcusPeters  160.0    9.0
 26       BobbyMcCain  156.0   14.0
 10     SenquezGolson  156.0    3.0
 30          TyeSmith  152.0   16.0
 16        CraigMager  152.0    1.0
 34   JaCoreyShepherd  148.0   18.0
 13        DJounSmith  144.0    4.0
 42        RyanMurphy  140.0   24.0
 32      Tevi

In [21]:
relativeError

{'C': 1.4285714285714286,
 'DB': 10.956521739130435,
 'DE': 5.0476190476190474,
 'DT': 7.636363636363637,
 'FB': 1.5,
 'G': 5.157894736842105,
 'LB': 9.722222222222221,
 'QB': 2.5714285714285716,
 'RB': 4.444444444444445,
 'T': 5.8,
 'TE': 5.6,
 'WR': 6.411764705882353}

In [22]:
coefficients

{'C':                 feature   coefficient
 8            confPac-10  2.508618e+15
 20        confSouthland  2.048217e+15
 11              confMWC  1.965751e+15
 18         confBig West  1.950579e+15
 14              confInd  1.664492e+15
 4            confBig 12  1.376020e+15
 29             weightQ1  9.346308e+14
 13             confCUSA  7.798355e+14
 12              confWAC  7.334513e+14
 27  heightinchestotalQ4  5.589735e+14
 26  heightinchestotalQ3  5.589735e+14
 41           verticalQ3  3.425348e+14
 39           verticalQ1  3.425348e+14
 42           verticalQ4  3.425348e+14
 50           twentyssQ2  3.351824e+14
 49           twentyssQ1  3.351824e+14
 52           twentyssQ4  3.351824e+14
 3               confSEC  2.915175e+14
 10         confSun Belt  2.564917e+14
 5           confBig Ten  2.456463e+14
 59              broadQ1  2.055482e+14
 60              broadQ2  2.055482e+14
 22             confSWAC  1.912141e+14
 38         verticalNone  3.477948e+13
 36            forty

In [23]:
# plot prediction -> Is this even possible to visualize?
# for p in positions:
#     if len(xTrain[p]) > 1 and len(xTest[p]) > 1:
#         predictor.fit(np.array(xTrain[p]), np.array(yTrain[p]))
#         prediction = predictor.predict(np.array(xTest[p])) 
#         plt.scatter(np.array(xTest[p]), np.array(yTest[p]), color='black')
#         plt.plot(np.array(xTest[p]), np.array(prediction), color='blue', linewidth=3)
#     else:
#         print 'Not enough data for position: {}'.format(p)